The Notebook is based on Foley et al.  J. Chem. Phys. 143, 243104 (2015): 

We import all necessary python packages

In [ ]:
import os
import numpy as np 
import matplotlib.pyplot as plt
from prody import * #module to build Gaussian Network Models
from matplotlib.pylab import *
ion()

We re-call the GNM: 
$u(\mathbf r) = 1/2 \Gamma \sum_{i=1} ^n \sum_{i<j} ^n \theta_{ij}(\mathbf{r}_{ij}-\mathbf {r}_{ij}^0)$

We can re-write the above equation as the sum of three terms, one for each direction in x, y, and z:
$u(\mathbf r) = 1/2 \Gamma ((\mathbf x-\mathbf x_0)^T \kappa (\mathbf x-\mathbf x_0)+(\mathbf y-\mathbf y_0)^T \kappa (\mathbf y-\mathbf y_0)+(\mathbf z-\mathbf z_0)^T \kappa (\mathbf z-\mathbf z_0))$

To build the Kirchhoff Matrix ($\kappa$), we first import the .pdb structure of the Protein of interest and select the alpha Carbons

In [ ]:
ProteinName = 
ubi=parsePDB()
calphas=ubi.select()
#To Do: show selection

Now, we have to build the GNM:

In [ ]:
gnm=GNM()
r_cut=
#spring constant, default is 1.0 
gamma=
gnm.buildKirchhoff()
#To do: extract the Kirchhoff matrix from gnm

After we have built the Kirchhoff-Matrix, we can perform some analysis:
3.) We analzye hinge regions and the slowest mode

1.) We visualize the contact/Kirchhoff-Matrix

In [ ]:
showContactMap()
plt.show()

2.) We visualize the cross-correlation Matrix

In [ ]:
#Calculate normal modes
gnm.calcModes('all',zeros=False)
gnm.getCovariance().round(2)

In [ ]:
showCrossCorr(gnm)
plt.show()

Hinge regions are stretches of a protein that can be involved in the propagation of conformational changes e.g. from the active site to the rest of the protein structure, including other subunits. They could be small loops in between secondary structures.

3.) We analzye hinge regions and the slowest mode

In [ ]:
hinges = calcHinges()

In [ ]:
#Hinge sites in the slowest mode can be obtained by:
calcHinges(gnm[0])
#To Do: assign residue to hinge regions of the slowest mode
resnums = 
mode2_hinges = 


Now, we visualize the slow mode and the squared fluctuations with and hinges:

In [ ]:
showMode(gnm[0], hinges=True, zero=True)
plt.show()

In [ ]:
#square fluctuations
showSqFlucts()
plt.show()

# Coarse graining

We have to define the coarse grained resolution based on the mapping matrix:
$\mathbf R = \mathbf M(\mathbf r)$

In [ ]:
def CG_map_matrix(n_atoms,n_beads):
    partition = (n_atoms/n_beads)*np.ones((n_beads))
    CG_map = np.zeros((n_beads, n_atoms)) 
    for i in range (len(partition)):
        #ind_left: 1st atom that belongs to a specific bead
        ind_left = int(np.sum(partition[:i]))
        #ind_right: final atom that belongs to a specific bead
        ind_right = int(np.sum(partition[:i+1]))
        CG_map[i, ind_left:ind_right] = 1./partition[i] 
        
    return CG_map

Find all integers that divide natoms, useful if we want beads with equal numbers of atoms


In [ ]:
def find_divisors(natoms):
    i = 1
    divisors = []
    while i<=natoms : 
        if (natoms % i==0): 
            divisors.append(i) 
        i +=1
    return divisors[:]


Based on the above functions, we now define the mapping scheme:

In [ ]:
natoms=len(calphas)
#how many atoms are grouped together
n_atoms_per_bead= find_divisors(natoms)
#Number of beads
N=np.zeros(len(n_atoms_per_bead))
for i in range (0,len(n_atoms_per_bead)):
    N[i]=natoms/n_atoms_per_bead[i]
    CG_map=CG_map_matrix(natoms,int(N[i]))

# To Do: 

1.) What is the meaning of partition in CG_map_matrix?

2.) Check for 3 mapping schemes if the mapping matrix is constructed in the right way.

# The Coarse Grained Potential

The interaction between CG-particles is given by the so-called many-body Potential of Mean Force $W$.

$W$ is a free energy, which can be decomposed as: $\mathbf W (\mathbf R;T)= U_W (\mathbf R;T)-TS_W (\mathbf R;T)$ 

For an analytical model like the GNM, we can exactly compute $W(\mathbf R;T)$:

$U_W(\mathbf R)=1/2 \Gamma ((\mathbf X-\mathbf X_0)^T K (\mathbf X-\mathbf X_0)+(\mathbf Y-\mathbf Y_0)^T K (\mathbf Y-\mathbf Y_0)+(\mathbf Z-\mathbf Z_0)^T K (\mathbf Z-\mathbf Z_0)) + k_B T \cdot 3/2 \cdot (n-N)$

$K=(\mathbf M \kappa^{-1} \mathbf M^T)^{-1}$

$S_W(\mathbf R) = 3/2 \cdot (n-N) (1+\ln \frac{2 \pi}{\beta \Gamma V^ {2/3}}) +3/2 \ln \frac{T_K}{t_k}$

In [ ]:
#pseudoinverse of CG Kirchhoff matrix
def pseudoinverse(k):

# compute inverse of the Kirchoff matrix by removing zero mode

    ll, lv = np.linalg.eigh(k)
    #print(ll)
    k_inv = np.zeros(np.shape(lv))
    for i, eig in enumerate(ll):
        if i > 0:
            k_inv += 1./ll[i]*np.outer(lv[:,i],lv[:,i])
    return k_inv

In [ ]:
#CG Kirchhoff matrix
def define_k_CG(k, CG_map):
    
# compute Kirchoff matrix for CG model
    
    nbeads = np.shape(CG_map)[0]
    k_inv = pseudoinverse(k)
    
    JN = np.ones((nbeads,1))
#    np.eye Return a 2-D array with ones on the diagonal and zeros elsewhere.
    QN = np.eye(nbeads) - (1./nbeads)*np.dot(JN,JN.T)
    
    prod = np.dot(QN, np.dot(CG_map, np.dot(k_inv, np.dot(CG_map.T,QN))))
    k_CG = pseudoinverse(prod)
    
    #k_CG = pseudoinverse(np.dot(CG_map,np.dot(k_inv,CG_map.T)))
    
    return k_CG

In [ ]:
#compute S_W
def S_W(n_atoms,N_beads,k,CG_map,bgV=10.0):
    
    k_CG = define_k_CG(k, CG_map)
    ll_aa, lv_aa = np.linalg.eigh(k)
    ll_CG, lv_CG = np.linalg.eigh(k_CG)
    
    tk = np.prod(ll_aa[1:])/float(natoms)
    Tk = np.prod(ll_CG[1:])/float(nbeads)
    
    
    

In [ ]:
#compute U_W
def U_W(n_atoms,N_beads,k_molecule,CG_map,calphas):
    CG_coords=np.dot(CG_map,calphas.getCoords()) 
    
    
    

# To Do

Complete the functions to compute U, S and W

In [ ]:
#Compute many body PMF for all number of beads
T=300#K
k_B=0.008314#kJ/(mol*K)
MB_PMF=[] #includes beta:(1/k_B*T)
for i in range (len(N)):
    CG_map=
    S_CG=
    U_CG=
    W_CG=


We now visulazie $W$ as a function of the resolution:

In [ ]:
fig=plt.figure(figsize=(8,6))
fig, ax1 = plt.subplots()
ax1.scatter(N,MB_PMF,color='k',label='')   
ax1.set_xlabel(r'N',fontsize=30)
ax1.set_ylabel(r'$\beta$W',fontsize=30)
ax1.set_yscale('log')
ax1.set_xscale('log')
#ax1.legend(loc='best')
plt.show()

# To Do

Plot the entropy and the energy as a function of the resolution

# To Do

re-run the notebook with a different Protein: 1PGB.pdb